<a href="https://colab.research.google.com/github/martincastanonicolas-source/IA/blob/master/01-numpy-analisis/analisis_meteorologico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Análisis de Datos Meteorológicos con NumPy**


In [183]:
# DATASET
import numpy as np
np.random.seed(42)
n_estaciones, n_dias, n_horas, n_vars = 5, 30, 24, 6
datos = np.zeros((n_estaciones, n_dias, n_horas, n_vars))

# Temperatura base por estación + variación diaria + ruido
temp_base = np.array([8, 12, 14, 10, 6]).reshape(5,1,1)
hora = np.arange(24).reshape(1,1,24)
variacion_diaria = 5 * np.sin((hora - 6) * np.pi / 12)
datos[:,:,:,0] = temp_base + variacion_diaria + np.random.normal(0, 2, (5,30,24))

# Resto de variables (humedad, presión, viento, precipitación, radiación)
datos[:,:,:,1] = np.clip(70 - datos[:,:,:,0] + np.random.normal(0, 10, (5,30,24)), 30, 100)
datos[:,:,:,2] = 1013 + np.random.normal(0, 5, (5,30,24))
datos[:,:,:,3] = np.abs(np.random.normal(15, 10, (5,30,24)))
datos[:,:,:,4] = np.maximum(0, np.random.exponential(2, (5,30,24)))
radiacion_base = np.maximum(0, 400 * np.sin((hora - 6) * np.pi / 12))
datos[:,:,:,5] = np.clip(radiacion_base + np.random.normal(0, 50, (5,30,24)), 0, 800)

##Bloque 1: Exploración y acceso a datos

In [184]:
stations = ["Madrid-Retiro", "Barcelona-El Prat", "Sevilla-Aeropuerto", "Bilbao-Sondica", "Granada-Base Aérea"]

#Ejercicio 1. Verificación de estructura
#Muestra la forma (shape) del array, el número total de mediciones y el tipo de datos (dtype). ¿Cuántas mediciones individuales contiene el dataset?
print(f"Ejercicio 1:")
print(f"Forma de los datos: {datos.shape}")
print(f"Tipo de la colección: {datos.dtype}")
print(f"Número de registros: {datos.size}")


#Ejercicio 2. Extracción de series temporales
#Extrae la serie de temperaturas de Madrid-Retiro durante el día 15. El resultado debe ser un array de 24 elementos.
print(f"\n\nEjercicio 2:")
temps_madrid_15_days = np.round(datos[0, 14, :, 0], 2)
print(f"Temperatura en Madrid-Retiro el día 15 - {temps_madrid_15_days}")


#Ejercicio 3. Comparativa entre estaciones
#Extrae las temperaturas de todas las estaciones a las 12:00 del día 20. El resultado debe ser un array de 5 elementos.
print(f"\n\nEjercicio 3:")
print(f"Temperaturas por estación día 20 a las 12:00")
stations_temps = np.round(datos[:, 19, 11, 0], 2)
for i, station in enumerate(stations):
    print(f"{station}: {stations_temps[i]}°C")


#Ejercicio 4. Bloque de datos
#Extrae todos los datos de Barcelona durante la primera semana (días 0-6). ¿Cuál es la forma del array resultante?
print(f"\n\nEjercicio 4:")
barcelona_first_week = datos[1, 0:7, :, :]
print(f"Forma de los datos de Barcelona en la primera semana: {barcelona_first_week.shape}")

Ejercicio 1:
Forma de los datos: (5, 30, 24, 6)
Tipo de la colección: float64
Número de registros: 21600


Ejercicio 2:
Temperatura en Madrid-Retiro el día 15 - [ 1.35  2.53  4.5   3.34  3.86  7.19  8.49  8.28  9.56 12.    9.43 10.01
 11.56 12.4  12.95 14.49 12.22  8.97  7.96  4.7   5.46  3.89  4.32  1.52]


Ejercicio 3:
Temperaturas por estación día 20 a las 12:00
Madrid-Retiro: 14.34°C
Barcelona-El Prat: 16.38°C
Sevilla-Aeropuerto: 16.58°C
Bilbao-Sondica: 16.23°C
Granada-Base Aérea: 14.01°C


Ejercicio 4:
Forma de los datos de Barcelona en la primera semana: (7, 24, 6)


##Bloque 2: Estadística descriptiva


In [185]:
hours = ["00:00", "01:00", "02:00", "03:00", "04:00", "05:00", "06:00", "07:00",
         "08:00", "09:00", "10:00", "11:00", "12:00", "13:00", "14:00", "15:00",
         "16:00", "17:00", "18:00", "19:00", "20:00", "21:00", "22:00", "23:00"]

#Ejercicio 5. Temperatura media por estación
#Calcula la temperatura media de todo el mes para cada estación. El resultado debe ser un array de 5 elementos. ¿Qué estación es más cálida? ¿Cuál más fría?

print("Ejercicio 5:")
mean_temps_per_station = np.mean(datos[:, :, :, 0], axis=(1, 2))
mean_temps_per_station = np.round(mean_temps_per_station, 2)
mean_temps_per_station = np.sort(mean_temps_per_station)[::-1]

for i, station in enumerate(reversed(stations)):
    print(f"{station}: {mean_temps_per_station[i]}")


#Ejercicio 6. Perfil horario medio
#Calcula la temperatura media de todas las estaciones para cada hora del día (promediando todos los días). El resultado debe ser un array de
#24 elementos que muestre el perfil térmico diario típico.

#Versión sin agrupar horas
#mean_temps_stations_per_hour = np.round(np.mean(datos[:, :, :, 0], axis=(0, 1)), 2)
#for i, hour in enumerate(hours):
#    print(f"{hour}: {mean_temps_stations_per_hour[i]}")

print(f"\nEjercicio 6:")
mean_temps_stations_per_hour = np.mean(datos[:, :, :, 0], axis=(0, 1))
mean_temps_stations_per_hour = np.round(mean_temps_stations_per_hour, 2)

for i in range(0, len(hours), 4):
    linea = "\t".join([f"{hours[j]}: {mean_temps_stations_per_hour[j]}" for j in range(i, min(i + 4, len(hours)))])
    print(linea)


#Ejercicio 7. Variabilidad climática
#Calcula la desviación estándar de la temperatura para cada estación. ¿Qué estación presenta mayor variabilidad térmica?
print(f"\nEjercicio 7:")
std_temps_per_station = np.std(datos[:, :, :, 0], axis=(1, 2))
std_temps_per_station = np.round(std_temps_per_station, 2)
std_temps_per_station = np.sort(std_temps_per_station)[::-1]

for i, station in enumerate(reversed(stations)):
    print(f"{station}: {std_temps_per_station[i]}")


#Ejercicio 8. Extremos meteorológicos
#Encuentra la temperatura máxima y mínima registradas en todo el dataset. Indica en qué estación, día y hora se produjeron
#(pista: usa np.argmax/np.argmin y np.unravel_index).
print(f"\nEjercicio 8:")
max_temp = np.round(np.max(datos[:, :, :, 0]), 2)
idx_max_temp = np.argmax(datos[:, :, :, 0])

min_temp = np.round(np.min(datos[:, :, :, 0]), 2)
idx_min_temp = np.argmin(datos[:, :, :, 0])

max_array = np.unravel_index(idx_max_temp, datos[:, :, :, 0].shape)
min_array = np.unravel_index(idx_min_temp, datos[:, :, :, 0].shape)

print(f"La temperatura máxima fue de {max_temp}°C en la estación {stations[max_array[0]]} el dia {max_array[1]} a las {hours[max_array[2]]}")
print(f"La temperatura mínima fue de {min_temp}°C en la estación {stations[min_array[0]]} el dia {min_array[1]} a las {hours[min_array[2]]}")

Ejercicio 5:
Granada-Base Aérea: 14.05
Bilbao-Sondica: 12.18
Sevilla-Aeropuerto: 10.05
Barcelona-El Prat: 7.98
Madrid-Retiro: 6.0

Ejercicio 6:
00:00: 5.03	01:00: 5.16	02:00: 5.48	03:00: 6.51
04:00: 7.64	05:00: 9.0	06:00: 10.23	07:00: 11.22
08:00: 12.55	09:00: 13.55	10:00: 14.12	11:00: 14.96
12:00: 15.19	13:00: 14.87	14:00: 14.37	15:00: 13.68
16:00: 12.35	17:00: 11.69	18:00: 10.17	19:00: 8.84
20:00: 7.55	21:00: 6.14	22:00: 5.64	23:00: 5.32

Ejercicio 7:
Granada-Base Aérea: 4.11
Bilbao-Sondica: 4.11
Sevilla-Aeropuerto: 4.09
Barcelona-El Prat: 4.06
Madrid-Retiro: 4.02

Ejercicio 8:
La temperatura máxima fue de 25.11°C en la estación Sevilla-Aeropuerto el dia 21 a las 13:00
La temperatura mínima fue de -4.19°C en la estación Granada-Base Aérea el dia 23 a las 22:00
